# Inflation

## Load libraries and import data

In [1]:
# Load libraries
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px


In [2]:
# Import data
# conditioning for being in Colab or not
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    folder = '/content/drive/MyDrive/basics-books/repos/financial-edu/bbooks-financial-edu/code/data/'
else:
    folder = ''

#> Files
# FOI-prices: 2016-.../2025-...
# IPCA-weights: 2018/2025
# IPCA-prices:  2018-.../2025-...
# NIC-weights: 2018/2025
# NIC-prices:  2018-.../2025-...
filen = {
    'FOI-prices'  : folder+'monthly-FOI.xlsx' ,
    'IPCA-weights': folder+'Classificazione Ecoicop (4 cifre) (IT1,168_6_DF_DCSP_IPCA3_1,1.0).xlsx',
    'IPCA-prices' : folder+'Classificazione Ecoicop (4 cifre) (IT1,168_760_DF_DCSP_IPCA1B2015_1,1.0).xlsx',
    'NIC-weights' : folder+'Classificazione Ecoicop (5 cifre) (IT1,167_743_DF_DCSP_NIC3B2015_3,1.0).xlsx',
    'NIC-prices'  : folder+'Classificazione Ecoicop (5 cifre) (IT1,167_744_DF_DCSP_NIC1B2015_4,1.0).xlsx'
}


In [3]:
df = {
    'prices' : pd.read_excel(filen['IPCA-prices'], sheet_name='data', decimal=','),
    'weights': pd.read_excel(filen['IPCA-weights'], sheet_name='data', decimal=',')
}

for kdf, idf in df.items():
    idf.columns = idf.columns.str.strip()  # strip whitespaces
    idf = idf.set_index(['Tempo'])
    idf = idf.transpose()
    idf = idf.rename(columns={'index': 'Tempo'})


FileNotFoundError: [Errno 2] No such file or directory: 'Classificazione Ecoicop (4 cifre) (IT1,168_760_DF_DCSP_IPCA1B2015_1,1.0).xlsx'

In [47]:
df['prices'].head()


,Tempo,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,...,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
0,[00] Indice generale,100.6,100.1,102.4,102.9,103.2,103.4,102,101.8,103.5,...,123,123.4,123.3,123.4,122.4,122.5,124.4,124.9,124.8,125
1,[01] -- prodotti alimentari e bevande analcoli...,103.9,103,103.2,103.6,104.3,103.9,103.1,103.1,103,...,130.8,132.3,133.3,132.6,133.8,133.7,133.8,134.8,135.4,135.7
2,[011] Prodotti alimentari,104,103.2,103.4,103.7,104.5,104.2,103.2,103.2,103.2,...,131.2,132.9,133.8,133,134.1,134,134,134.9,135.5,..
3,[0111] Pane e cereali,101.6,100.6,101.1,101.8,101.4,102,101.7,102.1,101.3,...,127.2,127.6,127.8,127.8,128.5,128.2,128.4,129.2,129.3,..
4,[0112] Carni,102.9,102.4,102.6,102.9,102.7,102.8,102.8,102.7,103,...,125.6,125.9,126.7,127,127.9,128,128.7,129.5,130.2,..


In [48]:
df['weights'].head()


,Tempo,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,[00] Indice generale,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000
1,[01] -- prodotti alimentari e bevande analcoli...,175648,176326,175240,175418,173257,172583,205912,194554,181443,181801,181425
2,[011] Prodotti alimentari,162005,162805,161810,161903,159432,158644,189091,179008,166582,167112,166336
3,[0111] Pane e cereali,30036,30342,29853,29558,29717,29778,35767,33586,31994,31422,31513
4,[0112] Carni,41803,40944,40876,39914,39286,38162,45695,43159,39770,40253,40080


In [37]:

# Set 'Tempo' as index
# df = df.set_index('Tempo')

# Transpose the DataFrame
df_T = df.transpose()

# Create a figure
fig = go.Figure()

# Add a trace for each row (now a column in df_T)
for column in df_T.columns:
    fig.add_trace(go.Scatter(
        x=df_T.index,
        y=df_T[column],
        mode='lines',
        name=column[:50]
    ))

# Customize layout
fig.update_layout(
    title='Trend Over Time',
    xaxis_title='Time',
    yaxis_title='Value',
    # xaxis_tickangle=-45,
    # hovermode='x unified',
    # template='plotly_white',
    height=600,
    width=1000,
    # legend=dict(orientation="v", x=1.02, y=1)
)

fig.show()


In [97]:
#> Extract code and label
ddf = {}

ddf = pd.DataFrame()
ddf['code' ] = df['weights']['Tempo'].str.extract(r'(\[\d+\])')
ddf['label'] = df['weights']['Tempo'].str.strip()

#> Determine parent code
def get_parent_code(code):
    if code is None or pd.isna(code):
        return None
    code_str = str(code).strip('[]')
    if len(code_str) <= 2:
        return None  # no parent
    elif len(code_str) == 3:
        return f"[{code_str[:2]}]"
    elif len(code_str) == 4:
        return f"[{code_str[:3]}]"
    else:
        return None

#> Determine parent code
def get_value(code):
    if code is None or pd.isna(code):
        return None
    code_str = str(code).strip('[]')
    if len(code_str) == 4:
        return int(1)
    else:
        return int(0)

ddf['parent'] = ddf['code'].map( get_parent_code )
# ddf['value'] = ddf['code'].map( get_value )
ddf['value'] = df['weights']['2025'] / 1e4

#> Drop [00] Indice generale
ddf = ddf[ddf['code'] != '[00]']

ddf.head(30)

# # Step 4: Map parent code to label
# code_label_map = dict(zip(df['code'], df['label']))
# df['parent_label'] = df['parent_code'].map(code_label_map)

# # Step 5: Full parent id
# df['parent'] = df['parent_code'] + ' ' + df['parent_label']
# df['parent'] = df['parent'].where(df['parent_code'].notnull())  # top-level parent stays NaN

,code,label,parent,value
1,[01],[01] -- prodotti alimentari e bevande analcoliche,None,18.1425
2,[011],[011] Prodotti alimentari,[01],16.6336
3,[0111],[0111] Pane e cereali,[011],3.1513
4,[0112],[0112] Carni,[011],4.0080
5,[0113],[0113] Pesci e prodotti ittici,[011],1.2412
6,[0114],"[0114] Latte, formaggi e uova",[011],2.5141
7,[0115],[0115] Oli e grassi,[011],0.6514
8,[0116],[0116] Frutta,[011],1.4763
9,[0117],[0117] Vegetali,[011],2.4341
10,[0118],"[0118] Zucchero, confetture, miele, cioccolato...",[011],0.8321


In [98]:
fig = px.sunburst(
    ddf,
    names='code',
    parents='parent',
    values='value',
    hover_data = ['label'],
    title='IPCA weights - 2025',
    branchvalues="total"  # "toatal" or "remainder"
)
fig.show()

In [115]:

import plotly.graph_objects as go

years = [str(y) for y in range(2018, 2026)]  # list of year strings

for year in years:
   ddf[str(year)] = df['weights'][str(year)] / 1e4


# Create the initial figure for the first year
fig = go.Figure()

fig.add_trace(go.Sunburst(
    ids=ddf['code'],
    labels=ddf['code'],
    parents=ddf['parent'],
    values=ddf['2025'],  # initial year
    hoverinfo='label+value+text',
    branchvalues="total",  # "toatal" or "remainder"
    sort=False,
    text=ddf['label'],
))
fig.update_layout(title="IPCA weights - 2025")

# Create one frame per year
frames = []
for year in years:
    frames.append(go.Frame(
        data=[go.Sunburst(
            ids=ddf['code'],
            labels=ddf['code'],
            parents=ddf['parent'],
            values=ddf[year],
            branchvalues="total",  # "toatal" or "remainder"
            sort=False,
            text=ddf['label']
        )],
        name=year,
        layout=go.Layout(title_text=f"IPCA weights - {year}")
    ))

fig.frames = frames

# Add slider steps for each year
steps = []
for year in years:
    steps.append(dict(
        method='animate',
        args=[[year],  # frame name
              dict(mode='immediate',
                   frame=dict(duration=500, redraw=True),
                   transition=dict(duration=300))],
        label=year
    ))

# Layout with slider
fig.update_layout(
    width=800, height=800,
    margin=dict(t=50, l=0, r=0, b=0),
    sliders=[dict(
        active=years.index('2025'),
        currentvalue={"prefix": "Year: "},
        pad={"t": 50},
        steps=steps
    )],
)

fig.show()
